In [ ]:
# Setup and imports
import sys
import os
import warnings
warnings.filterwarnings('ignore')

# Add the project root to path
project_root = os.path.dirname(os.path.abspath('.'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print("Python version:", sys.version)
print("Working directory:", os.getcwd())

In [ ]:
# Install dependencies if needed
!pip install -q pandas numpy scikit-learn

In [ ]:
# Import benchmark components
from Benchmarking import (
    # Runner
    BenchmarkRunner, BenchmarkConfig, ModelConfig,
    run_kenya_clinical_benchmark,
    
    # Metrics
    ClinicalMetrics, BootstrapEvaluator,
    
    # Datasets
    TaskType, BenchmarkSample, BenchmarkDataset, KenyaClinicalDataLoader,
    
    # Retrieval
    ClinicalRetriever, ClinicalKnowledgeBase,
    
    # Tasks
    ClinicalReasoningTask, DiagnosticTask, MultiChoiceTask, RetrievalQATask
)

print("✅ All benchmark components imported successfully!")

---
## 1. Test Metrics Module

In [ ]:
# Initialize metrics
metrics = ClinicalMetrics()
print("ClinicalMetrics initialized")
print(f"  Sentence Transformers available: {metrics.st_available}")

In [ ]:
# Test ROUGE computation
prediction = "The patient likely has acute coronary syndrome. Recommend ECG and cardiac enzymes."
reference = "Possible acute coronary syndrome. DDX includes MI, unstable angina. Immediate ECG, cardiac enzymes recommended."

rouge_scores = metrics.compute_rouge(prediction, reference)
print("ROUGE Scores:")
for metric, score in rouge_scores.items():
    print(f"  {metric}: {score:.4f}")

In [ ]:
# Test semantic similarity
semantic_sim = metrics.compute_semantic_similarity(prediction, reference)
print(f"\nSemantic Similarity: {semantic_sim:.4f}")

In [ ]:
# Test DDX accuracy
predicted_ddx = ["Acute coronary syndrome", "Myocardial infarction", "Unstable angina", "GERD"]
true_ddx = ["Acute coronary syndrome", "MI", "Pneumonia"]

ddx_metrics = metrics.compute_ddx_accuracy(predicted_ddx, true_ddx)
print("\nDDX Accuracy Metrics:")
for metric, score in ddx_metrics.items():
    print(f"  {metric}: {score:.4f}")

In [ ]:
# Test clinical completeness
response = """
ASSESSMENT: The patient presents with classic signs of acute coronary syndrome.
DIFFERENTIAL DIAGNOSIS: 1) STEMI, 2) NSTEMI, 3) Unstable angina
INVESTIGATIONS: ECG, Troponin, Chest X-ray
MANAGEMENT: Aspirin, Heparin, Consider PCI
FOLLOW-UP: Cardiology referral within 24 hours
"""

completeness = metrics.compute_clinical_completeness(
    response, 
    required_elements=['assessment', 'diagnosis', 'investigation', 'management', 'follow-up']
)
print(f"\nClinical Completeness Score: {completeness:.4f}")

---
## 2. Test Dataset Loading

In [ ]:
import pandas as pd

# Check if data file exists
data_path = "train.csv"
if os.path.exists(data_path):
    df = pd.read_csv(data_path)
    print(f"✅ Loaded training data: {len(df)} samples")
    print(f"\nColumns: {list(df.columns)}")
    print(f"\nSample prompt (truncated):")
    print(df['Prompt'].iloc[0][:500] + "...")
else:
    print(f"❌ Data file not found at: {data_path}")

In [ ]:
# Initialize data loader
loader = KenyaClinicalDataLoader(
    train_path="train.csv",
    val_split=0.2,
    random_seed=42
)

print(f"\n✅ KenyaClinicalDataLoader initialized")
print(f"  Total samples: {len(loader.train_df)}")
print(f"  Train samples: {len(loader.train_df) - int(len(loader.train_df) * 0.2)}")
print(f"  Val samples: {int(len(loader.train_df) * 0.2)}")

In [ ]:
# Create Clinical Reasoning dataset
clinical_dataset = loader.create_clinical_reasoning_dataset()

print(f"\n✅ Clinical Reasoning Dataset Created")
print(f"  Name: {clinical_dataset.name}")
print(f"  Task Type: {clinical_dataset.task_type}")
print(f"  Samples: {len(clinical_dataset.samples)}")

# Show first sample
if clinical_dataset.samples:
    sample = clinical_dataset.samples[0]
    print(f"\n  First sample:")
    print(f"    ID: {sample.id}")
    print(f"    Input (truncated): {sample.input_text[:200]}...")
    print(f"    Expected output (truncated): {sample.expected_output[:200]}...")

In [ ]:
# Create Diagnostic dataset
diagnostic_dataset = loader.create_diagnostic_dataset()

print(f"\n✅ Diagnostic Dataset Created")
print(f"  Samples: {len(diagnostic_dataset.samples)}")

if diagnostic_dataset.samples:
    sample = diagnostic_dataset.samples[0]
    print(f"  First sample metadata: {sample.metadata.get('ddx_snomed', 'N/A')[:100] if sample.metadata.get('ddx_snomed') else 'N/A'}")

In [ ]:
# Create Multi-Choice dataset
mc_dataset = loader.create_multi_choice_dataset()

print(f"\n✅ Multi-Choice Dataset Created")
print(f"  Samples: {len(mc_dataset.samples)}")

if mc_dataset.samples:
    sample = mc_dataset.samples[0]
    print(f"  First sample choices: {sample.metadata.get('choices', [])}")

---
## 3. Test Retrieval Module

In [ ]:
# Initialize retriever
retriever = ClinicalRetriever()

# Add sample documents
sample_docs = [
    {"id": "doc1", "text": "Management of diabetic ketoacidosis involves IV fluids, insulin therapy, and electrolyte monitoring."},
    {"id": "doc2", "text": "Bacterial meningitis requires empirical antibiotics, lumbar puncture for diagnosis, and ICU admission."},
    {"id": "doc3", "text": "Malaria treatment in Kenya follows WHO guidelines with artemisinin-based combination therapy."},
    {"id": "doc4", "text": "Hypertensive emergency management includes IV labetalol or nitroprusside with continuous monitoring."},
    {"id": "doc5", "text": "Pediatric pneumonia assessment includes respiratory rate, chest indrawing, and oxygen saturation."},
    {"id": "doc6", "text": "HIV treatment follows Kenya national guidelines with first-line ART including TDF/3TC/DTG."},
    {"id": "doc7", "text": "Tuberculosis diagnosis uses GeneXpert MTB/RIF for rapid detection of rifampicin resistance."},
]

retriever.add_documents(sample_docs)
print(f"✅ Added {len(sample_docs)} documents to retriever")

In [ ]:
# Test TF-IDF retrieval
query = "How to manage a diabetic patient with high blood sugar and acidosis?"

results = retriever.retrieve_tfidf(query, top_k=3)

print(f"\n🔍 Query: {query}")
print(f"\nTF-IDF Retrieval Results:")
for i, r in enumerate(results, 1):
    print(f"  {i}. [{r.doc_id}] Score: {r.score:.4f}")
    print(f"     {r.text[:80]}...")

In [ ]:
# Test retrieval evaluation
queries = [
    "diabetic ketoacidosis management",
    "meningitis treatment antibiotics",
    "malaria treatment guidelines kenya"
]
relevant_docs = [["doc1"], ["doc2"], ["doc3"]]

eval_results = retriever.evaluate_retrieval(
    queries, relevant_docs, method="tfidf", k_values=[1, 3, 5]
)

print("\n📊 Retrieval Evaluation:")
print(f"  MRR: {eval_results['mrr']:.4f}")
for k in [1, 3, 5]:
    print(f"  Recall@{k}: {eval_results.get(f'recall@{k}', 0):.4f}")

---
## 4. Test Task Evaluators

In [ ]:
# Test Clinical Reasoning Task
cr_task = ClinicalReasoningTask()

sample = BenchmarkSample(
    id="test_cr_001",
    input_text="A 45-year-old male farmer from Kisumu presents with fever, headache, and jaundice for 3 days.",
    expected_output="DDX includes malaria, hepatitis, leptospirosis. Recommend malaria RDT, LFTs, urinalysis. Start empirical antimalarials pending results.",
    task_type=TaskType.CLINICAL_REASONING,
    metadata={'county': 'Kisumu', 'facility': 'County Hospital'}
)

prediction = "Patient may have malaria or hepatitis. Recommend malaria test and liver function tests."

result = cr_task.evaluate_sample(sample, prediction)

print("\n📋 Clinical Reasoning Task Evaluation:")
print(f"  Sample ID: {result.sample_id}")
print(f"  Metrics:")
for metric, value in result.metrics.items():
    print(f"    {metric}: {value:.4f}")

In [ ]:
# Test Diagnostic Task
diag_task = DiagnosticTask()

sample = BenchmarkSample(
    id="test_diag_001",
    input_text="Child with high fever, neck stiffness, and photophobia.",
    expected_output="1. Bacterial meningitis\n2. Viral meningitis\n3. Subarachnoid hemorrhage",
    task_type=TaskType.DIAGNOSTIC,
    metadata={}
)

prediction = "1. Meningitis\n2. Encephalitis\n3. Brain abscess\n4. Malaria"

result = diag_task.evaluate_sample(sample, prediction)

print("\n🩺 Diagnostic Task Evaluation:")
print(f"  Sample ID: {result.sample_id}")
print(f"  Predicted DDX: {result.metadata.get('predicted_ddx', [])}")
print(f"  True DDX: {result.metadata.get('true_ddx', [])}")
print(f"  Metrics:")
for metric, value in result.metrics.items():
    print(f"    {metric}: {value:.4f}")

In [ ]:
# Test Multi-Choice Task
mc_task = MultiChoiceTask()

sample = BenchmarkSample(
    id="test_mc_001",
    input_text="A patient presents with productive cough and fever. What is the most appropriate initial investigation?",
    expected_output="B",
    task_type=TaskType.MULTI_CHOICE,
    metadata={
        'choices': ["CT scan", "Chest X-ray", "MRI", "PET scan"],
        'correct_answer': 'B'
    }
)

# Format prompt with choices
prompt = mc_task.format_prompt(sample)
print("📝 Multi-Choice Prompt:")
print(prompt)

prediction = "The answer is B. Chest X-ray is the most appropriate initial investigation."
result = mc_task.evaluate_sample(sample, prediction)

print(f"\n✅ Multi-Choice Task Evaluation:")
print(f"  Predicted: {result.prediction}")
print(f"  Correct: {result.ground_truth}")
print(f"  Accuracy: {result.metrics['accuracy']}")

---
## 5. Test Benchmark Runner

In [ ]:
# Create benchmark configuration
config = BenchmarkConfig(
    name="kenya_clinical_demo",
    version="1.0.0",
    description="Demo benchmark run",
    tasks=["clinical_reasoning"],  # Start with one task
    train_data_path="train.csv",
    val_split=0.2,
    output_dir="benchmark_results",
    save_predictions=True
)

print("✅ BenchmarkConfig created")
print(f"  Name: {config.name}")
print(f"  Tasks: {config.tasks}")

In [ ]:
# Initialize runner
runner = BenchmarkRunner(config=config, data_dir=".")

# Setup (load data, create datasets)
runner.setup()

In [ ]:
# Define a simple baseline model (echo model for testing)
def baseline_model(prompt: str) -> str:
    """Simple baseline that extracts key clinical terms."""
    # This is just for testing - replace with actual model
    response = """
    Based on the clinical presentation:
    
    ASSESSMENT:
    Patient requires urgent evaluation.
    
    DIFFERENTIAL DIAGNOSIS:
    1. Consider infectious etiology
    2. Metabolic causes
    3. Structural abnormalities
    
    INVESTIGATIONS:
    - Complete blood count
    - Basic metabolic panel
    - Relevant imaging
    
    MANAGEMENT:
    - Supportive care
    - Treat underlying cause
    - Monitor closely
    
    FOLLOW-UP:
    - Review in 24-48 hours
    - Specialist referral if needed
    """
    return response

# Create model config
model_config = ModelConfig(
    name="baseline_model",
    model_fn=baseline_model,
    description="Simple baseline for testing"
)

print("✅ Baseline model configured")

In [ ]:
# Run evaluation
print("Running benchmark evaluation...")
result = runner.evaluate_model(model_config)

In [ ]:
# View results
print("\n" + "="*60)
print("BENCHMARK RESULTS")
print("="*60)
print(f"\nModel: {result.model_name}")
print(f"Aggregate Score: {result.aggregate_score:.4f}")
print(f"Timestamp: {result.timestamp}")

for task_name, task_result in result.task_results.items():
    print(f"\n--- {task_name} ---")
    print(f"Samples: {task_result.num_samples}")
    print("Key Metrics:")
    for metric, value in sorted(task_result.aggregate_metrics.items()):
        if 'mean' in metric:
            print(f"  {metric}: {value:.4f}")

In [ ]:
# Generate markdown report
report = runner.generate_report()
print(report)

---
## 6. Test with Multiple Models Comparison

In [ ]:
# Define another model for comparison
def improved_model(prompt: str) -> str:
    """Slightly better baseline."""
    return """
    CLINICAL ASSESSMENT:
    The patient presents with concerning symptoms requiring systematic evaluation.
    
    DIFFERENTIAL DIAGNOSIS:
    1. Primary consideration based on presentation
    2. Secondary differential
    3. Must rule out emergent conditions
    4. Consider endemic diseases (malaria, TB)
    
    RECOMMENDED INVESTIGATIONS:
    - Complete blood count with differential
    - Comprehensive metabolic panel
    - Malaria RDT (if febrile)
    - Urinalysis
    - Chest X-ray if respiratory symptoms
    
    MANAGEMENT PLAN:
    - IV access and fluids if dehydrated
    - Empirical antimalarials if indicated
    - Symptomatic treatment
    - Close monitoring
    
    DISPOSITION AND FOLLOW-UP:
    - Admit if unstable
    - Referral to specialist if needed
    - Follow up within 48 hours
    """

# Evaluate second model
model_config2 = ModelConfig(
    name="improved_model",
    model_fn=improved_model,
    description="Improved baseline"
)

result2 = runner.evaluate_model(model_config2)

In [ ]:
# Compare models
comparison = runner.compare_models()

print("\n📊 MODEL COMPARISON")
print("="*60)
print(f"\nRanking (by aggregate score):")
for i, model in enumerate(comparison['ranking'], 1):
    score = comparison['aggregate_scores'][model]
    print(f"  {i}. {model}: {score:.4f}")

In [ ]:
# Final report with both models
final_report = runner.generate_report()
print(final_report)

---
## 7. Summary & Next Steps

### ✅ Completed
1. **Metrics Module** - ROUGE, semantic similarity, DDX accuracy, clinical completeness
2. **Datasets Module** - Clinical reasoning, diagnostic, multi-choice, retrieval QA datasets
3. **Retrieval Module** - TF-IDF retrieval with evaluation
4. **Tasks Module** - All four task types with evaluators
5. **Benchmark Runner** - Full orchestration and reporting

### 🔜 Next Steps
1. **Add real models** - Integrate FLAN-T5, GPT-4, Gemini evaluations
2. **Enable semantic retrieval** - Install sentence-transformers for dense retrieval
3. **Expand knowledge base** - Add Kenya clinical guidelines
4. **Add more metrics** - BERTScore, BLEU, clinical F1
5. **Create leaderboard** - Track model performance over time

In [ ]:
print("\n🎉 Benchmark framework test completed successfully!")
print("\nFramework Structure:")
print("  Benchmarking/")
print("  ├── __init__.py      - Module exports")
print("  ├── metrics.py       - Evaluation metrics")
print("  ├── datasets.py      - Dataset loading")
print("  ├── retrieval.py     - Knowledge retrieval")
print("  ├── tasks.py         - Task evaluators")
print("  └── benchmark_runner.py - Main orchestration")